In [1]:
# qcnn_classifier.py

"""
Quantum Convolutional Neural Network (QCNN) for Nepali Numerals Classification.

This script implements a QCNN using PennyLane and PyTorch to classify Nepali numerals (0-9)
based on 28x28 pixel grayscale images. The dataset should be organized with images in
subdirectories per class.

Directory Structure:
your_project/
├── qcnn_classifier.py
└── data/
    └── numerals/
        ├── 0/
        │   ├── image1.png
        │   ├── image2.png
        │   └── ...
        ├── 1/
        │   ├── image1.png
        │   ├── image2.png
        │   └── ...
        └── ...
        └── 9/
            ├── image1.png
            ├── image2.png
            └── ...
"""

import os
import logging
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from matplotlib import pyplot as plt
from PIL import Image
from tqdm import tqdm  # For progress bars
import pennylane as qml
from pennylane import numpy as pnp

# ==============================
# Logging Setup
# ==============================
def setup_logging(log_file='training.log'):
    """
    Sets up logging to output to both console and a file with a specific format.
    Args:
        log_file (str): Filename for the log file.
    """
    logger = logging.getLogger('QCNNClassifier')
    logger.setLevel(logging.INFO)

    # Create handlers
    c_handler = logging.StreamHandler()
    f_handler = logging.FileHandler(log_file, mode='w')

    c_handler.setLevel(logging.INFO)
    f_handler.setLevel(logging.INFO)

    # Create formatter and add it to handlers
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
    c_handler.setFormatter(formatter)
    f_handler.setFormatter(formatter)

    # Add handlers to the logger
    if not logger.handlers:
        logger.addHandler(c_handler)
        logger.addHandler(f_handler)

    return logger

logger = setup_logging()

# ==============================
# Dataset Definition
# ==============================
class NepaliMNISTDataset(Dataset):
    def __init__(self, root_dir, transform=None, patch_size=4):
        """
        Args:
            root_dir (string): Directory with all the images organized in subdirectories per class.
            transform (callable, optional): Optional transform to be applied on a sample.
            patch_size (int): Size of each patch (patch_size x patch_size).
        """
        self.root_dir = root_dir
        self.transform = transform
        self.patch_size = patch_size
        self.data = []
        self.labels = []

        logger.info(f"Loading dataset from directory: {root_dir}")
        for label in range(10):
            label_dir = os.path.join(root_dir, str(label))
            if not os.path.isdir(label_dir):
                logger.warning(f"Directory for label {label} does not exist: {label_dir}")
                continue
            for img_file in os.listdir(label_dir):
                img_path = os.path.join(label_dir, img_file)
                if os.path.isfile(img_path) and img_file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                    self.data.append(img_path)
                    self.labels.append(label)
        logger.info(f"Loaded {len(self.data)} samples.")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = Image.open(self.data[idx]).convert('L')  # Convert to grayscale
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# ==============================
# Data Transformations
# ==============================
# Define the transformation pipeline for the dataset
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((28, 28)),  # Resize to 28x28 pixels
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# ==============================
# Data Loaders Initialization
# ==============================
ROOT_DIR = "/Users/sahajrajmalla/Documents/nepali-quantum-mnist/data/numerals"  # Update this path as needed

def initialize_data_loaders(root_dir, batch_size=16, patch_size=4):
    """
    Initializes and returns the training and testing data loaders.
    Args:
        root_dir (str): Root directory of the dataset.
        batch_size (int): Number of samples per batch.
        patch_size (int): Size of each image patch.
    Returns:
        Tuple[DataLoader, DataLoader]: Training and testing data loaders.
    """
    logger.info("Initializing Data Loaders")
    dataset = NepaliMNISTDataset(root_dir=root_dir, transform=transform, patch_size=patch_size)
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=0,  # Set to 0 to avoid multiprocessing issues in scripts or notebooks
        pin_memory=True  # Optional: can speed up data transfer to GPU
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=0,  # Set to 0 to avoid multiprocessing issues in scripts or notebooks
        pin_memory=True  # Optional
    )
    logger.info("Data Loaders initialized successfully: %d training samples, %d testing samples", train_size, test_size)
    return train_loader, test_loader

# ==============================
# QCNN Layers Definitions
# ==============================
def get_qcnn_layers(n_qubits=4, depth=1):
    """
    Defines the QCNN layers using PennyLane templates.
    Args:
        n_qubits (int): Number of qubits per patch.
        depth (int): Number of convolutional layers.
    Returns:
        list: List of QCNN layers, each layer is a list of tuples (gate_type, wires).
    """
    layers = []
    for _ in range(depth):
        conv_layer = []
        for i in range(n_qubits):
            conv_layer.append(('Rot', i))  # Parameterized gate: Rot
        # Entangling layer
        for i in range(n_qubits - 1):
            conv_layer.append(('CNOT', [i, i + 1]))  # Non-parameterized gate: CNOT
        layers.append(conv_layer)
    return layers

# ==============================
# Quantum Convolutional Layer
# ==============================
class QuantumConvolution(nn.Module):
    def __init__(self, n_qubits=4, depth=1):
        super(QuantumConvolution, self).__init__()
        self.n_qubits = n_qubits
        self.depth = depth
        self.layers = get_qcnn_layers(n_qubits, depth)
        # Initialize parameters: for each Rot gate, three angles (phi, theta, omega)
        # The number of Rot gates per layer is n_qubits
        # For CNOT gates, parameters are unused but need to maintain the structure
        self.params = nn.Parameter(0.01 * torch.randn(len(self.layers), len(self.layers[0]), 3))

    def forward(self, inputs):
        """
        Forward pass through the quantum convolutional layer.
        Args:
            inputs (Tensor): Input tensor of shape (batch_size, n_patches, n_qubits)
        Returns:
            Tensor: Output tensor of shape (batch_size, n_patches, n_qubits)
        """
        batch_size, n_patches, _ = inputs.shape  # inputs shape: (batch_size, n_patches, n_qubits)
        outputs = []
        for b in range(batch_size):
            patch_outputs = []
            for p in range(n_patches):
                patch = inputs[b, p]
                q_out = qnode(self.params, self.layers, patch)  # Pass entire params
                q_out = torch.tensor(q_out, dtype=torch.float32, device=patch.device)  # Ensure float32
                patch_outputs.append(q_out)
            # Stack patch outputs: shape (n_patches, n_qubits)
            outputs.append(torch.stack(patch_outputs))
        # Stack all quantum outputs: shape (batch_size, n_patches, n_qubits)
        return torch.stack(outputs)

# ==============================
# Quantum Node Definition
# ==============================
n_qubits_conv = 4  # Number of qubits per patch
dev_conv = qml.device("default.qubit", wires=n_qubits_conv)

@qml.qnode(dev_conv, interface='torch', diff_method='parameter-shift')
def qnode(params, layers, inputs):
    """
    Defines a quantum circuit for convolutional layers.
    Args:
        params (Tensor): Parameters for rotation gates, shape (n_layers, n_gates_per_layer, 3).
        layers (list): List of QCNN layers, each layer is a list of tuples (gate_type, wires).
        inputs (Tensor): Input data encoded into the circuit, shape (n_qubits,).
    Returns:
        Tensor: Measurement results from all qubits.
    """
    # Amplitude Encoding
    qml.AmplitudeEmbedding(inputs, wires=range(n_qubits_conv), normalize=True, pad_with=0.0)

    # Apply QCNN layers
    for layer, param_layer in zip(layers, params):
        for gate_info, param in zip(layer, param_layer):
            gate_type, wires = gate_info
            if gate_type == 'Rot':
                phi, theta, omega = param
                qml.Rot(phi, theta, omega, wires=wires)
            elif gate_type == 'CNOT':
                qml.CNOT(wires=wires)
            else:
                raise ValueError(f"Unknown gate type: {gate_type}")

    # Measurement: Expectation value of PauliZ on all qubits
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits_conv)]

# ==============================
# Quantum Pooling Layer
# ==============================
class QuantumPooling(nn.Module):
    def __init__(self, n_qubits=4):
        super(QuantumPooling, self).__init__()
        self.n_qubits = n_qubits

    def forward(self, inputs):
        """
        Forward pass through the quantum pooling layer.
        Args:
            inputs (Tensor): Input tensor of shape (batch_size, n_patches, n_qubits)
        Returns:
            Tensor: Pooled tensor of shape (batch_size, n_patches, 1)
        """
        # Simple pooling: average over qubits
        return torch.mean(inputs, dim=2, keepdim=True)  # Shape: (batch_size, n_patches, 1)

# ==============================
# Quantum Fully Connected Layer
# ==============================
class QuantumFullyConnected(nn.Module):
    def __init__(self, input_dim=49, output_dim=10):
        super(QuantumFullyConnected, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)  # Mapping to 10 classes

    def forward(self, inputs):
        """
        Forward pass through the quantum fully connected layer.
        Args:
            inputs (Tensor): Input tensor of shape (batch_size, n_patches, n_qubits)
        Returns:
            Tensor: Output tensor of shape (batch_size, 10)
        """
        batch_size = inputs.shape[0]
        x = inputs.view(batch_size, -1)  # Shape: (batch_size, input_dim)
        out = self.fc(x)  # Shape: (batch_size, output_dim)
        return out

# ==============================
# QCNN Model Definition
# ==============================
class QCNN(nn.Module):
    def __init__(self, n_qubits=4, depth=1, patch_size=4):
        super(QCNN, self).__init__()
        self.n_qubits = n_qubits
        self.depth = depth
        self.patch_size = patch_size
        self.quantum_conv = QuantumConvolution(n_qubits=self.n_qubits, depth=self.depth)
        self.pool = QuantumPooling(n_qubits=1)
        self.quantum_fc = QuantumFullyConnected(input_dim=49, output_dim=10)  # Updated input_dim

    def forward(self, x):
        """
        Forward pass through the QCNN model.
        Args:
            x (Tensor): Input tensor of shape (batch_size, 1, 28, 28)
        Returns:
            Tensor: Output tensor of shape (batch_size, 10)
        """
        batch_size = x.shape[0]
        # Divide image into patches using unfold
        patches = x.unfold(2, self.patch_size, self.patch_size).unfold(3, self.patch_size, self.patch_size)
        # patches shape: (batch_size, channels, n_patches_h, n_patches_w, patch_size, patch_size)
        patches = patches.contiguous().view(batch_size, -1, self.patch_size * self.patch_size)  # (batch_size, n_patches, 16)
        # Normalize patches
        patches = patches / patches.norm(dim=2, keepdim=True)
        # Pass through quantum convolutional layer
        conv_out = self.quantum_conv(patches)  # (batch_size, n_patches, n_qubits)
        # Pass through pooling layer
        pooled_out = self.pool(conv_out)  # (batch_size, n_patches, 1)
        # Pass through fully connected layer
        fc_out = self.quantum_fc(pooled_out)  # (batch_size, 10)
        return fc_out

# ==============================
# Training Function
# ==============================
def train_model(model, train_loader, optimizer, loss_fn, epochs, device):
    """
    Trains the model for a specified number of epochs.
    Args:
        model (nn.Module): The neural network model.
        train_loader (DataLoader): DataLoader for training data.
        optimizer (torch.optim.Optimizer): Optimizer for training.
        loss_fn (nn.Module): Loss function.
        epochs (int): Number of training epochs.
        device (torch.device): Device to run the training on.
    """
    logger.info("Starting training for %d epochs", epochs)
    model.train()
    for epoch in range(1, epochs + 1):
        epoch_loss = 0.0
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch}/{epochs}', leave=False)
        for images, labels in progress_bar:
            images = images.to(device)  # Shape: (batch_size, 1, 28, 28)
            labels = labels.long().to(device)

            optimizer.zero_grad()
            outputs = model(images)  # Shape: (batch_size, 10)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            progress_bar.set_postfix({'Loss': loss.item()})
        avg_loss = epoch_loss / len(train_loader)
        logger.info(f"Epoch {epoch}/{epochs} - Average Loss: {avg_loss:.4f}")

# ==============================
# Evaluation Function
# ==============================
def evaluate_model(model, test_loader, device):
    """
    Evaluates the model on the test dataset and returns the accuracy.
    Args:
        model (nn.Module): The neural network model.
        test_loader (DataLoader): DataLoader for testing data.
        device (torch.device): Device to run the evaluation on.
    Returns:
        float: Accuracy percentage.
    """
    logger.info("Starting evaluation")
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc='Evaluating', leave=False):
            images = images.to(device)
            labels = labels.long().to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    logger.info(f"Evaluation completed with Accuracy: {accuracy:.2f}%")
    return accuracy

# ==============================
# Visualization Function
# ==============================
def visualize_predictions(model, test_loader, device, num_images=6):
    """
    Visualizes a few predictions from the model.
    Args:
        model (nn.Module): The neural network model.
        test_loader (DataLoader): DataLoader for testing data.
        device (torch.device): Device to run the visualization on.
        num_images (int): Number of images to visualize.
    """
    logger.info("Visualizing predictions")
    model.eval()
    images, labels = next(iter(test_loader))
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)
    _, predictions = torch.max(outputs, 1)

    fig, axes = plt.subplots(1, num_images, figsize=(15, 5))
    for idx, ax in enumerate(axes):
        if idx >= len(images):
            break
        # Reshape the image to 28x28
        image = images[idx].cpu().numpy().reshape(28, 28)
        ax.imshow(image, cmap='gray')
        ax.set_title(f"True: {labels[idx].item()}\nPred: {predictions[idx].item()}")
        ax.axis('off')
    plt.tight_layout()
    plt.show()
    logger.info("Visualization complete")

# ==============================
# Main Execution
# ==============================
def main():
    logger.info("Initializing the Quantum Convolutional Neural Network (QCNN)")
    train_loader, test_loader = initialize_data_loaders(ROOT_DIR, batch_size=16, patch_size=4)

    model = QCNN(n_qubits=4, depth=1, patch_size=4)  # You can increase depth for more complex models
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.005)  # Adjust learning rate as needed
    loss_fn = nn.CrossEntropyLoss()

    logger.info("Model architecture:\n%s", model)
    logger.info("Starting training process")
    train_model(model, train_loader, optimizer, loss_fn, epochs=20, device=device)  # Adjust epochs as needed

    logger.info("Starting evaluation on test data")
    accuracy = evaluate_model(model, test_loader, device=device)
    logger.info(f"Test Accuracy: {accuracy:.2f}%")

    logger.info("Starting visualization of predictions")
    visualize_predictions(model, test_loader, device=device)

    logger.info("Quantum Convolutional Neural Network execution completed")

if __name__ == "__main__":
    main()


2024-12-24 11:02:35,368 - INFO - Initializing the Quantum Convolutional Neural Network (QCNN)
2024-12-24 11:02:35,369 - INFO - Initializing Data Loaders
2024-12-24 11:02:35,369 - INFO - Loading dataset from directory: /Users/sahajrajmalla/Documents/nepali-quantum-mnist/data/numerals
2024-12-24 11:02:35,400 - INFO - Loaded 2880 samples.
2024-12-24 11:02:35,413 - INFO - Data Loaders initialized successfully: 2304 training samples, 576 testing samples
2024-12-24 11:02:35,427 - INFO - Model architecture:
QCNN(
  (quantum_conv): QuantumConvolution()
  (pool): QuantumPooling()
  (quantum_fc): QuantumFullyConnected(
    (fc): Linear(in_features=49, out_features=10, bias=True)
  )
)
2024-12-24 11:02:35,428 - INFO - Starting training process
2024-12-24 11:02:35,428 - INFO - Starting training for 20 epochs


KeyboardInterrupt: 

In [ ]:
# 2024-12-24 10:06:13,298 - INFO - Initializing the Quantum Convolutional Neural Network (QCNN)
# 2024-12-24 10:06:13,298 - INFO - Initializing Data Loaders
# 2024-12-24 10:06:13,299 - INFO - Loading dataset from directory: /Users/sahajrajmalla/Documents/nepali-quantum-mnist/data/numerals
# 2024-12-24 10:06:13,324 - INFO - Loaded 2880 samples.
# 2024-12-24 10:06:13,335 - INFO - Data Loaders initialized successfully: 2304 training samples, 576 testing samples
# 2024-12-24 10:06:13,350 - INFO - Model architecture:
# QCNN(
#   (quantum_conv): QuantumConvolution()
#   (pool): QuantumPooling()
#   (quantum_fc): QuantumFullyConnected(
#     (fc): Linear(in_features=49, out_features=10, bias=True)
#   )
# )
# 2024-12-24 10:06:13,351 - INFO - Starting training process
# 2024-12-24 10:06:13,352 - INFO - Starting training for 20 epochs
# 2024-12-24 10:08:55,178 - INFO - Epoch 1/20 - Average Loss: 2.2502      
# 2024-12-24 10:11:29,459 - INFO - Epoch 2/20 - Average Loss: 2.1320      
# 2024-12-24 10:14:04,471 - INFO - Epoch 3/20 - Average Loss: 2.0383      
# 2024-12-24 10:16:33,356 - INFO - Epoch 4/20 - Average Loss: 1.9627      
# 2024-12-24 10:19:02,626 - INFO - Epoch 5/20 - Average Loss: 1.9000      
# 2024-12-24 10:21:35,039 - INFO - Epoch 6/20 - Average Loss: 1.8490      
# 2024-12-24 10:24:22,218 - INFO - Epoch 7/20 - Average Loss: 1.8062      
# 2024-12-24 10:27:35,139 - INFO - Epoch 8/20 - Average Loss: 1.7706      
# 2024-12-24 10:30:27,833 - INFO - Epoch 9/20 - Average Loss: 1.7396      
# 2024-12-24 10:32:55,537 - INFO - Epoch 10/20 - Average Loss: 1.7135      
# ...
# 2024-12-24 11:00:51,124 - INFO - Evaluation completed with Accuracy: 45.31%
# 2024-12-24 11:00:51,126 - INFO - Test Accuracy: 45.31%
# 2024-12-24 11:00:51,126 - INFO - Starting visualization of predictions
# 2024-12-24 11:00:51,127 - INFO - Visualizing predictions